In [83]:
import pandas as pd
import numpy as np
import codecs
import sys

In [140]:
stop_words = [
    "از",
    "این",
    "آن",
    "به",
    "با",
    "بر",
    "برای",
    "پس",
    "تا",
    "در",
    "را",
    "که",
    "و",
]

punctuations = [
    ".",
    "،",
    "؛",
    ":",
    "؟",
    "!",
    "»",
    "«",
]
def read_dataset(path, name):
    df = pd.read_excel(path + name)
    return df

def delete_punctuations(doc):
    punctuations = '،:؛؟!»«()[]"*,{.}@!?'
    edited_doc = doc.translate(str.maketrans('', '', punctuations))
    return edited_doc

def tokenize(df):
    content = df.content

    tokens = []
    for i in range(0, content.size):
        doc = content[i]
        doc = delete_punctuations(doc)
        tokenized_doc = doc.split()
        for token in tokenized_doc:
            temp = []
            temp.append(token)
            temp.append(df.id[i])
            tokens.append(temp)
    tokens.sort()
    return tokens

def create_inverted_indexes(tokens):
    inverted_indexes = []
    doc_temp = []
    token_temp = ""
    for token in tokens:
        if token[0] == token_temp:
            doc_temp.append(token[1])
        else:
            temp = []
            temp.append(token_temp)
            doc_temp = set(doc_temp)
            temp.append(doc_temp)
            inverted_indexes.append(temp)
            doc_temp = []

        token_temp = token[0]
        # if len(doc_temp) > 0 and token[1] != doc_temp[-1]:
        doc_temp.append(token[1])
    
    return inverted_indexes

def search_token(inverted_indexes, token_name):
    for token in inverted_indexes:
        if token[0] == token_name:
            return token[1]
    print("this token not exist in our database")


In [136]:
df = read_dataset("datasets/", "IR_Spring2021_ph12_7k.xlsx")


In [141]:
tokens = tokenize(df)
inverted_indexes = create_inverted_indexes(tokens)


In [142]:
for i in range(5000, 5100):
    print(inverted_indexes[i][0])
    print(inverted_indexes[i][1])

ارتوپد
{6739, 6741}
ارتوپدی
{5760, 6020, 6022, 5754, 6013, 6015}
ارتوپدیhttp//98032209825
{6672, 6674, 6669}
ارتکاب
{3078, 4425, 1776, 1809, 50, 661, 4921, 1112, 2969, 2714, 3101}
ارث
{3425, 6755, 4390, 6758, 6953, 3376, 4727}
ارثی
{6946, 6947, 6949, 6887, 6890, 6609, 2451, 6196, 5781, 6643, 6943, 6648, 6873, 6879}
ارثیه
{4047}
ارث‌
{3150}
ارج
{6208, 6017, 6941, 911, 6937, 6009, 2138, 3099, 6781, 6494}
ارجاع
{4608, 3588, 5637, 3078, 2057, 4105, 5646, 4113, 5137, 5652, 6678, 2584, 6681, 5658, 2595, 2596, 2086, 3111, 3112, 4142, 2607, 5172, 2110, 65, 2114, 6215, 3144, 3657, 2122, 2123, 2638, 4177, 2132, 2644, 4183, 6232, 2144, 2656, 2657, 2662, 2160, 2681, 4217, 5754, 4222, 5758, 2176, 2688, 5248, 5760, 5766, 2183, 3208, 6791, 3722, 6797, 3726, 2191, 2193, 4754, 2201, 2204, 2211, 4773, 2726, 5799, 6824, 3754, 6827, 2732, 6830, 5808, 6834, 5813, 2745, 3770, 5817, 2757, 1737, 2256, 2258, 2770, 3283, 4822, 3800, 2265, 6360, 6874, 2268, 4319, 3296, 5855, 6880, 5352, 1770, 3819, 3824, 6385, 2

In [143]:

search_token(inverted_indexes, "اجتماعی")

{27,
 43,
 65,
 95,
 110,
 120,
 124,
 188,
 215,
 216,
 249,
 261,
 297,
 325,
 397,
 402,
 436,
 442,
 486,
 505,
 506,
 590,
 629,
 780,
 781,
 845,
 867,
 919,
 942,
 992,
 1003,
 1110,
 1276,
 1291,
 1344,
 1366,
 1398,
 1482,
 1509,
 1532,
 1610,
 1734,
 1753,
 1766,
 1769,
 1775,
 1804,
 1815,
 1837,
 1844,
 1859,
 1863,
 1868,
 1876,
 1880,
 1882,
 1887,
 1904,
 1905,
 1907,
 1921,
 1928,
 1932,
 1948,
 1950,
 1958,
 1961,
 1970,
 1982,
 1983,
 1984,
 2020,
 2024,
 2025,
 2030,
 2038,
 2041,
 2042,
 2053,
 2055,
 2057,
 2063,
 2069,
 2072,
 2075,
 2078,
 2080,
 2086,
 2090,
 2094,
 2099,
 2101,
 2135,
 2136,
 2137,
 2138,
 2147,
 2148,
 2159,
 2165,
 2167,
 2196,
 2219,
 2221,
 2222,
 2224,
 2226,
 2230,
 2239,
 2240,
 2243,
 2248,
 2264,
 2280,
 2285,
 2288,
 2298,
 2300,
 2323,
 2328,
 2341,
 2352,
 2353,
 2354,
 2365,
 2368,
 2383,
 2387,
 2403,
 2416,
 2430,
 2438,
 2440,
 2441,
 2457,
 2477,
 2484,
 2485,
 2534,
 2537,
 2541,
 2543,
 2576,
 2587,
 2588,
 2597,
 2606,
 2607

In [224]:
punctuations = [
    ".",
    "،",
    "؛",
    ":",
    "؟",
    "!",
    "»",
    "«",
]

import re

tweet = "I am tired! I like fruit...and milk"
s = "هی.پسر؟ چطوری تو. من، عالیی!!!!!"
s1 = "باگ با بیدار ترینا،خوب تر،باغ ها"


clean = re.sub(r"""
               ["،:؛؟!»«()[*,{.}@!? ]+  # Accept one or more copies of punctuation
               \ *           # plus zero or more copies of a space,
               """,
               " ",          # and replace it with a single space
               s1, flags=re.VERBOSE)

stop_words = [
    "از",
    "این",
    "آن",
    "به",
    "با",
    "بر",
    "برای",
    "پس",
    "تا",
    "در",
    "را",
    "که",
    "و",
    "ترین"
]

for s in stop_words:
    my_regex = r"\b"+s+r"\b"
    clean = re.sub(my_regex , "", clean)

print(s1 + "\n" + clean)

p = '،:؛؟!»«()[]"*,{.}@!?'
s = "هی.پسر؟ چطوری تو. من، عالیی!!!!!"
s1 = "بیدارترین،خوب تر،باغ ها"



# x = s.translate(str.maketrans('', '', p))
print(x)


# for token in doc:
#     for p in punctuations:
#         if token == p:
#             token.replace("geeks", "Geeks")


باگ با بیدار ترینا،خوب تر،باغ ها
باگ  بیدار ترینا خوب تر باغ ها
هیپسر چطوری تو من عالیی


In [144]:
my_list = [1,1,2,3,2,2,4,5,6,2,1]
my_final_list = set(my_list)
print(list(my_final_list))

[1, 2, 3, 4, 5, 6]
